In [22]:
query_redis_dependencies = '''
select p.name, p.alias, p.leader,
       i.name, i.pom, i.type,
       d.groupId, d.artifactId, d.version
from dependencies d
right join items i on d.item_id = i.id
right join projects p on i.project_id = p.id
where i.is_online = 1 and i.is_offline = 0 and p.is_online = 1 and p.is_offline = 0
and (d.artifactId = 'jedis' or d.artifactId = 'lettuce-core' or d.artifactId = 'richgo-redis-api' or d.artifactId = 'spring-boot-starter-data-redis')
order by p.leader desc , p.name, i.name, artifactId;
'''

import pymysql
conn = pymysql.connect(host='172.16.162.211', port=3306, user='root', password='password', database='project',
                           charset='utf8mb4')
cursor = conn.cursor()
cursor.execute(query_redis_dependencies)
results = cursor.fetchall()
cursor.close()
conn.close()


In [23]:
items = {}
for result in results:

    if items.get(result[3]) is None :
        item = {
            'project': result[0],
            'alias': result[1],
            'leader': result[2],
            'name': result[3],
            'pom': result[4],
            'type': result[5],
        }
        items[result[3]] = item
    else:
        item = items[result[3]]

    if result[7] == 'jedis':
        item['jedis'] = result[8]
    elif result[7] == 'lettuce-core':
        item['lettuce-core'] = result[8]
    elif result[7] == 'richgo-redis-api':
        item['richgo-redis-api'] = result[8]
    elif result[7] == 'spring-boot-starter-data-redis':
        item['spring-boot-starter-data-redis'] = result[8]

In [24]:
import openpyxl
book = openpyxl.Workbook()
sheet = book.create_sheet('Sheet1',0)
sheet.column_dimensions['A'].width = 22
sheet.column_dimensions['B'].width = 15
sheet.column_dimensions['C'].width = 10
sheet.column_dimensions['D'].width = 35
sheet.column_dimensions['E'].width = 45
sheet.column_dimensions['F'].width = 9
sheet.column_dimensions['G'].width = 7
sheet.column_dimensions['H'].width = 17
sheet.column_dimensions['I'].width = 17
sheet.column_dimensions['J'].width = 15
sheet.column_dimensions['K'].width = 40
titles = ['project','alias','leader', 'name', 'pom', 'type', 'jedis', 'lettuce-core', 'richgo-redis-api', 'spring-boot-starter-data-redis', '备注']
for index, title in enumerate(titles):
    sheet.cell(1, index + 1, title)
for index, item in enumerate(items):
    sheet.cell(index + 2, 1, items[item].get('project'))
    sheet.cell(index + 2, 2, items[item].get('alias'))
    sheet.cell(index + 2, 3, items[item].get('leader'))
    sheet.cell(index + 2, 4, items[item].get('name'))
    sheet.cell(index + 2, 5, items[item].get('pom'))
    sheet.cell(index + 2, 6, items[item].get('type'))
    sheet.cell(index + 2, 7, items[item].get('jedis'))
    sheet.cell(index + 2, 8, items[item].get('lettuce-core'))
    sheet.cell(index + 2, 9, items[item].get('richgo-redis-api'))
    sheet.cell(index + 2, 10, items[item].get('spring-boot-starter-data-redis'))
    message = []
    if items[item].get('type') == 'pom' or items[item].get('type') == 'api' or items[item].get('type') == 'common' :
        message.append('非运行项目')
    if items[item].get('jedis') is not None and items[item].get('richgo-redis-api') is not None:
        message.append('使用RedisClusterUtil')
    if items[item].get('lettuce-core') is not None and items[item].get('spring-boot-starter-data-redis') is not None:
        message.append('使用RedisTemplate')
    sheet.cell(index + 2, 11, ','.join(message))

book.save('redis.xlsx')
